In [ ]:
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Flatten, Dense, Lambda
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception # TensorFlow ONLY
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from tqdm import tqdm 

In [ ]:
## set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
train_image_path = './training set/'
test_image_path = './testing set/'

In [ ]:
image_list = []
import os
def get_data(Dir):
    X = []
    y = []
    for nextDir in os.listdir(Dir):
        if not nextDir.startswith('.'):
            if nextDir in ['controlled']:
                label = 0
            elif nextDir in ['uncontrolled']:
                label = 1
                
            temp = Dir + nextDir
                
            for file in tqdm(os.listdir(temp)):
                image = load_img(temp + '/' + file,target_size=(256,256))
                image = img_to_array(image)
                image /= 255
                X.append(image)
                y.append(label)
                image_list.append(file)   
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [ ]:
x_train,y_train = get_data(train_image_path)
x_test,y_test = get_data(test_image_path)

In [ ]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train, 7)
Y_test = to_categorical(y_test, 7)

In [ ]:
import efficientnet.tfkeras as efn 
efnmodel = efn.EfficientNetB5(include_top=True, weights=None, input_tensor=None, input_shape=(256,256,3))  # or weights='noisy-student'
efnmodel.load_weights('./weights/efficientnet-b5_weights_tf_dim_ordering_tf_kernels.h5')
x = efnmodel.layers[-2].output
output_layer = Dense(2, activation='softmax', name='fc2')(x)

# combine the original model with the new output layer
efn2 = Model(inputs=efnmodel.input, outputs=output_layer)

# compile the new model
efn2.compile(optimizer=Adam(lr=0.0008), # lr
                loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = 'efficientnet.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
hist = efn2.fit(x_train, Y_train, 
                 batch_size=32, 
                 epochs=100, 
                 verbose=1,
                 callbacks=[checkpoint],
                 validation_data = (x_test , Y_test),
                 shuffle =True)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['savefig.dpi'] = 500
plt.rcParams['figure.dpi'] = 500
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for accuracy
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()